In [4]:
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt

In [5]:
user = "jeroendereu"
password = "***********"

In [6]:
api = SentinelAPI(user, password, "https://scihub.copernicus.eu/dhus")

In [105]:
api.download("8dd7d819-d02d-42f9-a645-c161851ba2da")

Downloading: 100%|██████████████████████████| 483M/483M [06:27<00:00, 1.54MB/s]


{'date': datetime.datetime(2017, 3, 30, 10, 56, 21, 26000),
 'downloaded_bytes': 483267019,
 'footprint': 'POLYGON((3.878085340859089 51.445265877430046,3.811160567513305 51.31587185266105,3.737128263614465 51.17179727196963,3.663254134732998 51.027598036949335,3.590018293627545 50.88343913481603,3.517122259906812 50.739215231097845,3.444484482098213 50.594938238327,3.378073272626582 50.46128492214912,2.999718211669176 50.46379840175528,2.999712175851301 51.45118220579359,3.878085340859089 51.445265877430046,3.878085340859089 51.445265877430046))',
 'id': '8dd7d819-d02d-42f9-a645-c161851ba2da',
 'md5': '46D6FE01003FA02C1DA0693DB77D9B48',
 'path': '.\\S2A_MSIL2A_20170330T105621_N0204_R094_T31UES_20170330T110334.zip',
 'size': 483267019,
 'title': 'S2A_MSIL2A_20170330T105621_N0204_R094_T31UES_20170330T110334',
 'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('8dd7d819-d02d-42f9-a645-c161851ba2da')/$value"}

In [106]:
api.get_product_odata("8dd7d819-d02d-42f9-a645-c161851ba2da")

{'date': datetime.datetime(2017, 3, 30, 10, 56, 21, 26000),
 'footprint': 'POLYGON((3.878085340859089 51.445265877430046,3.811160567513305 51.31587185266105,3.737128263614465 51.17179727196963,3.663254134732998 51.027598036949335,3.590018293627545 50.88343913481603,3.517122259906812 50.739215231097845,3.444484482098213 50.594938238327,3.378073272626582 50.46128492214912,2.999718211669176 50.46379840175528,2.999712175851301 51.45118220579359,3.878085340859089 51.445265877430046,3.878085340859089 51.445265877430046))',
 'id': '8dd7d819-d02d-42f9-a645-c161851ba2da',
 'md5': '46D6FE01003FA02C1DA0693DB77D9B48',
 'size': 483267019,
 'title': 'S2A_MSIL2A_20170330T105621_N0204_R094_T31UES_20170330T110334',
 'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('8dd7d819-d02d-42f9-a645-c161851ba2da')/$value"}

In [107]:
api.get_product_odata("8dd7d819-d02d-42f9-a645-c161851ba2da", full=True)

{'Aot retrieval accuracy': 0.0,
 'Bare soils percentage': 25.730711,
 'Cloud cover percentage': 2.187012,
 'Cloud shadow percentage': 0.094218,
 'Dark features percentage': 2.214143,
 'Date': datetime.datetime(2017, 3, 30, 10, 56, 21, 26000),
 'Degraded MSI data percentage': 0,
 'Degraded ancillary data percentage': 0,
 'Filename': 'S2A_MSIL2A_20170330T105621_N0204_R094_T31UES_20170330T110334.SAFE',
 'Footprint': '<gml:Polygon srsName="http://www.opengis.net/gml/srs/epsg.xml#4326" xmlns:gml="http://www.opengis.net/gml">\n   <gml:outerBoundaryIs>\n      <gml:LinearRing>\n         <gml:coordinates>51.445265877430046,3.878085340859089 51.31587185266105,3.811160567513305 51.17179727196963,3.737128263614465 51.027598036949335,3.663254134732998 50.88343913481603,3.590018293627545 50.739215231097845,3.517122259906812 50.594938238327,3.444484482098213 50.46128492214912,3.378073272626582 50.46379840175528,2.999718211669176 51.45118220579359,2.999712175851301 51.445265877430046,3.878085340859089

In [29]:
from geojson import Polygon
import matplotlib.pyplot as plt
%matplotlib inline

In [66]:
footprint = geojson_to_wkt(read_geojson("C:/WERKMAP/Anaconda/sentinelsat/Sentinelsat.geojson"))

In [70]:
footprint

'POLYGON ((3.5550000 51.1800000, 3.6500000 51.1800000, 3.6500000 51.1150000, 3.5550000 51.1150000, 3.5550000 51.1800000))'

In [103]:
products = api.query(footprint, '20170101', 'NOW', platformname = 'Sentinel-2', processinglevel = 'Level-2Ap', cloudcoverpercentage = '[0 TO 30]')

In [104]:
products

OrderedDict([('48728576-5565-4594-900d-084a7d806212',
              {'baresoilpercentage': 21.818556,
               'beginposition': datetime.datetime(2017, 5, 29, 10, 56, 21, 26000),
               'cloudcoverpercentage': 16.381376,
               'endposition': datetime.datetime(2017, 5, 29, 10, 56, 21, 26000),
               'filename': 'S2A_MSIL2A_20170529T105621_N0205_R094_T31UES_20170529T110523.SAFE',
               'footprint': 'POLYGON ((3.862833210684212 51.445368608950034,3.811772021738506 51.346545787368974,3.737544046256364 51.20245922243156,3.663655612245543 51.05826903501925,3.590183728062112 50.913947670667646,3.517124839920729 50.769608563959295,3.44469709115523 50.6252486700904,3.372683447611854 50.48098760882046,3.362944564494567 50.461385424833715,2.999718211669176 50.46379840175528,2.999712175851301 51.45118220579359,3.862833210684212 51.445368608950034))',
               'format': 'SAFE',
               'gmlfootprint': '<gml:Polygon srsName="http://www.opengis.net